# Generate TikTok *#hashtags* with OpenAI 

In this project we will create a classification using OpenAI to generate hashtags for tiktok descriptions.

In the *TODO* list we have:

1.   Access credentials
2.   Get data from TikTok
3.   Prepare data for classification
4.   Test the classification

The Youtube tutorial is available [here](https://).

References:

1.   [Classification OpenAI API Docs](https://beta.openai.com/docs/guides/classifications)
2.   [TikTok API Wrapper](https://github.com/davidteather/TikTok-Api)  
3.   [Async API](https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook)
4.   ['TikTokApi' object has no attribute 'region'](https://github.com/davidteather/TikTok-Api/issues/672
)
5.   [Selenium and Google Colab](https://github.com/davidteather/TikTok-Api/issues/548
)
6.   [byHashtag not working anymore](https://github.com/davidteather/TikTok-Api/issues/631)
7.   [TikTok sent invalid JSON back](https://github.com/davidteather/TikTok-Api/issues/392)
8.   [David's Tutorial](https://www.youtube.com/watch?v=zwLmLfVI-VQ)
9.   [Hashtags](https://influencermarketinghub.com/top-tiktok-hashtags/)
10.  [Playwright](https://playwright.dev/docs/intro/)

#Setup

In [1]:
PATH = r'<DIR_DRIVE>'

## Install libraries

In [ ]:
!pip install TikTokApi
!pip install python-dotenv
!pip install --upgrade jsonlines
!pip install --upgrade openai
!pip install google
!python -m pip install playwright
!apt install libgstreamer1.0-0 libwoff1 libgstreamer1.0-0 libgstreamer1.0-dev libgstreamer-plugins-good1.0-0 libgstreamer-gl1.0-0 libgstreamer-plugins-bad1.0-0

In [ ]:
!playwright install

In [ ]:
!playwright install-deps

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

*load environment variables*

In [ ]:
from dotenv import load_dotenv

load_dotenv(PATH+'.env')

## Import libraries

In [4]:
import os
import jsonlines

## Asyncio

In [5]:
import asyncio
from ipykernel.eventloops import register_integration

@register_integration('asyncio')
def loop_asyncio(kernel):
    '''Start a kernel with asyncio event loop support.'''
    loop = asyncio.get_event_loop()

    def kernel_handler():
        loop.call_soon(kernel.do_one_iteration)
        loop.call_later(kernel._poll_interval, kernel_handler)

    loop.call_soon(kernel_handler)
    try:
        if not loop.is_running():
            loop.run_forever()
    finally:
        loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()


#Prepare data from TikTok

## Hashtags

In [ ]:
%%writefile tags.py

hashtags = [
              "#tiktok",
              "#love",
              "#like",
              "#follow",
              "#explore"
              "#meme",
              "#video",
              "#followforfollowback",
              "#foryoupage",
              "#fyp",
              "#foryou",
              "#viral",
              "#funny",
              "#followme",
              "#cute",
              "#fun",
              "#music",
              "#happy",
              "#fashion",
              "#comedy",
              "#bestvideo",
              "#tiktok4fun",
              "#thisis4u",
              "#loveyoutiktok",
    
]

## Get data by hashtag

In [ ]:
%%writefile main.py

from TikTokApi import TikTokApi
from tags import *
import jsonlines
import os

api = TikTokApi.get_instance(custom_verifyFp=os.getenv("TIKTOK_COOKIE"), use_test_endpoints=True)
jsons = []
count = 100

for tag in hashtags:
  try:
    tiktoks = api.by_hashtag(tag, count=count)
    for tiktok in tiktoks:
      jsons.append({
          "text": tiktok['desc'],
          "label": tag[1:],
          "metadata": {
              "source": "tiktok.com"
          }

      })
  except:
    pass

print(len(jsons))

with jsonlines.open('tiktok.jsonl', 'w') as writer:
  writer.write_all(jsons)

*run main.py*

In [ ]:
!python main.py

# Classify post and find hashtags with OpenAI

## Create classification from jsonl

In [ ]:
import openai
openai.File.create(file=open("tiktok.jsonl"), purpose="classifications")

## Test classification

In [ ]:
openai.Classification.create(
    file="<YOUR_FILE_MODEL>",
    query="<YOUR_PROMPT>",
    search_model="ada", 
    model="curie", 
    max_examples=3
)